In [ ]:
#Import Dependencies
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
session.query(measurement.date).order_by(measurement.date.desc()).first()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
session.query(measurement.date).\
    filter(measurement.date > '2016-08-23').order_by(measurement.date).all()

# Perform a query to retrieve the data and precipitation scores
sel = [measurement.date, func.avg(measurement.prcp)]

Yr_query = session.query(*sel).\
    filter(measurement.date > '2016-08-23').group_by(measurement.date).order_by(measurement.date).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
Yr_query_df = pd.DataFrame(Yr_query, columns = ['date', 'prcp'])
Yr_query_df.set_index('date', inplace =True)

# Sort the dataframe by date
Yr_query_df.sort_values(by='date', ascending=False)

Yr_query_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

Yr_query_df.plot(rot=90, figsize=(10, 10))
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
Yr_query_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
session.query(measurement).group_by(measurement.station).count()

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
q1 = session.query(station.station)
q1.all()

In [ ]:
USC00519397 = session.query(measurement.station).filter(measurement.station == 'USC00519397').count()
USC00513117 = session.query(measurement.station).filter(measurement.station == 'USC00513117').count()
USC00514830 = session.query(measurement.station).filter(measurement.station == 'USC00514830').count()
USC00517948 = session.query(measurement.station).filter(measurement.station == 'USC00517948').count()
USC00518838 = session.query(measurement.station).filter(measurement.station == 'USC00518838').count()
USC00519523 = session.query(measurement.station).filter(measurement.station == 'USC00519523').count()
USC00519281 = session.query(measurement.station).filter(measurement.station == 'USC00519281').count()
USC00511918 = session.query(measurement.station).filter(measurement.station == 'USC00511918').count()
USC00516128 = session.query(measurement.station).filter(measurement.station == 'USC00516128').count()

print(f' USC00519397 has {USC00519397} reports')
print(f' USC00513117 has {USC00513117} reports')
print(f' USC00514830 has {USC00514830} reports')
print(f' USC00517948 has {USC00517948} reports')
print(f' USC00518838 has {USC00518838} reports')
print(f' USC00519523 has {USC00519523} reports')
print(f' USC00519281 has {USC00519281} reports')
print(f' USC00511918 has {USC00511918} reports')
print(f' USC00516128 has {USC00516128} reports')

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
sel = [func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)]

data = session.query(*sel).group_by(measurement.station).order_by(func.count(measurement.id).desc()).first()

data

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

data_ = session.query(measurement.date, measurement.tobs).filter(measurement.station == "USC00519281").\
filter(measurement.date > '2016-08-23').order_by(measurement.date.desc()).all()

hist_df = pd.DataFrame(data_, columns= ['Date', 'tobs'])
hist_df.set_index('Date', inplace=True)
hist = hist_df.hist(bins=12)
hist = hist[0]
for x in hist:
    x.set_title('Last 12 Months of Temperature Observations from Station: USC00519281')
    x.set_xlabel('Temperature')
    x.set_ylabel('Frequency')

# Close session

In [ ]:
# Close Session
session.close()